In [1]:
import cv2
from deepface import DeepFace
import os
import random
import numpy as np
import matplotlib.pyplot as plt
pairs=500
data_path='VGG-Face2/data'

In [12]:
def load_image_pairs(image_pairs):
    data = []
    labels = []
    for img1_path, img2_path, label in image_pairs:        
        data.append((img1_path, img2_path))
        labels.append(label)
    return data, labels

def create_pairs(data_path,pairs):
    positive_pairs = []
    negative_pairs = []
    testset=os.listdir(data_path)
    for person in testset:
        path=data_path+"/"+person
        images=os.listdir(path)
        if len(images)<2:
            continue
        for _ in range(pairs//len(testset)):
            img1,img2=random.sample(images,2)
            positive_pairs.append((path+"/"+img1, path+"/"+img2, 1))
    for _ in range(pairs):
        person1,person2=random.sample(testset, 2)
        person1_path = os.path.join(data_path, person1)
        person2_path = os.path.join(data_path, person2)
        img1=random.choice(os.listdir(person1_path))
        img2=random.choice(os.listdir(person2_path))
        negative_pairs.append((person1_path+"/"+img1, person2_path+"/"+img2, 0))
    return positive_pairs+negative_pairs

def evaluate_model(model_name, X_test, y_test):
    predictions = []
    for i in range(len(X_test)):
        img1_path=X_test[i][0]
        img2_path=X_test[i][1]
        print(img1_path,img2_path)
        try:
            result = DeepFace.verify(img1_path, img2_path, model_name=model_name)
            predictions.append(int(result['verified']))
        except Exception as e:
            print(f"Exception while processing images in DeepFace: {e}")
            predictions.append(0)
        
    
    accuracy = np.mean(np.array(predictions) == y_test)
    return accuracy

In [ ]:
random.seed(42)
image_pairs=create_pairs(data_path,pairs)
print(image_pairs)
data,labels=load_image_pairs(image_pairs)
model_name=["OpenFace", "VGG-Face", "DeepID"]
a=[]
for model in model_name:
    accuracy=evaluate_model(model,data,labels)
    a.append(accuracy)

In [32]:
data_to_save=list((model_name,a))
print(data_to_save)
with open('seed_42.txt','w') as f:
    f.write(str(data_to_save))

[['OpenFace', 'VGG-Face', 'DeepID'], [0.5655555555555556, 0.8233333333333334, 0.5866666666666667]]
